# Assignment 1 - Big Data
**Tsiky Tafita RAKOTOHERISOA**

# Question 1: Loading a large CSV

**dataset: activity_log_raw.csv**

**Task 1:  Write a function which returns the date range in the dataset (i.e, earliest and oldest). Please call your function and print out the date range.**

**Task 2:  Write a function which returns the year and month  with the largest number of events.**

**Loading the dataset from the drive and process it**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
file_path = ('/content/drive/MyDrive/DATASETS/raw/activity_log_raw.csv')
out_path = ('/test_data.csv')

**Data cleaning**

In [4]:
def clean_csv(file_path, output_path):

    with open(file_path, "r", encoding="ISO-8859-1", errors="ignore") as infile, \
         open(output_path, "w", encoding="ISO-8859-1") as outfile:
        for line in infile:
            # Remove null bytes and write to the cleaned file
            cleaned_line = line.replace("\0", "")
            outfile.write(cleaned_line)

In [5]:
clean_csv(file_path, out_path)
data = pd.read_csv(out_path, encoding="ISO-8859-1", on_bad_lines='skip', chunksize=10000)

**Extract the column activity time and doing task 1 and task 2 in the same function**

*This function extract the activity time cilumn and transform it to format date time. It creates two new columns which are year and month that contains year and month. It returns the earliest date, latest date, year and month with largest number of events*

In [6]:
def extract_activity_time(df):
    # Convert ACTIVITY_TIME to datetime
    df["ACTIVITY_TIME"] = pd.to_datetime(
        df["ACTIVITY_TIME"],
        format="%d-%b-%y %I.%M.%S.%f %p",
        errors="coerce"  # Invalid formats are set as NaT
    )

    # Drop rows where ACTIVITY_TIME is NaT (invalid datetime)
    df = df.dropna(subset=["ACTIVITY_TIME"])

    if df.empty:
        return {
            "earliest_date": None,
            "latest_date": None,
            "most_frequent_year": None,
            "most_frequent_year_count": None,
            "most_frequent_month": None,
            "most_frequent_month_count": None
        }

    # Extract Year and Month
    df['Year'] = df['ACTIVITY_TIME'].dt.year
    df['Month'] = df['ACTIVITY_TIME'].dt.month

    # Get the year and month with the largest counts
    year_counts = df['Year'].value_counts()
    month_counts = df['Month'].value_counts()

    most_frequent_year = year_counts.idxmax() if not year_counts.empty else None
    most_frequent_year_count = year_counts.max() if not year_counts.empty else None
    most_frequent_month = month_counts.idxmax() if not month_counts.empty else None
    most_frequent_month_count = month_counts.max() if not month_counts.empty else None

    # Get the earliest and latest dates
    earliest_date = df["ACTIVITY_TIME"].max()
    latest_date = df["ACTIVITY_TIME"].min()

    return {
        "earliest_date": earliest_date,
        "latest_date": latest_date,
        "most_frequent_year": most_frequent_year,
        "most_frequent_year_count": most_frequent_year_count,
        "most_frequent_month": most_frequent_month,
        "most_frequent_month_count": most_frequent_month_count
    }


**Make the return of the function in a list after transform as a dataframe**

In [7]:
results = []
for df in data:
    results.append(extract_activity_time(df))

results_df = pd.DataFrame(results)

<ipython-input-6-0d9a38f53c90>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['ACTIVITY_TIME'].dt.year
<ipython-input-6-0d9a38f53c90>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['ACTIVITY_TIME'].dt.month
<ipython-input-6-0d9a38f53c90>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [8]:
results_df.head()

,earliest_date,latest_date,most_frequent_year,most_frequent_year_count,most_frequent_month,most_frequent_month_count
0,2015-04-13 22:39:00.431,2015-04-13 19:54:15.530,2015.0,10000.0,4.0,10000.0
1,2015-04-13 20:02:42.360,2015-04-13 16:29:06.260,2015.0,10000.0,4.0,10000.0
2,2015-04-13 19:25:53.880,2015-04-13 14:01:36.230,2015.0,10000.0,4.0,10000.0
3,2015-04-13 16:29:06.260,2015-04-13 12:57:00.020,2015.0,10000.0,4.0,10000.0
4,2015-04-13 14:01:36.230,2015-04-13 12:42:40.510,2015.0,10000.0,4.0,10000.0


**Now, we can work on the dataframe, we an easily extract the earliest date, latest date , year and month with the largest number of events for the wole dataset**

In [9]:
earliest_date = results_df['earliest_date'].max()
latest_date = results_df['latest_date'].min()
max_year = results_df.groupby('most_frequent_year')['most_frequent_year_count'].sum().idxmax()
max_month = results_df.groupby('most_frequent_month')['most_frequent_month_count'].sum().idxmax()


print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)
print("Max Year:", max_year)
print("Max Month:", max_month)

Earliest Date: 2015-04-13 22:39:00.431000
Latest Date: 2012-08-15 20:01:36.621000
Max Year: 2013.0
Max Month: 3.0


**Task-3. Describe what strategy you used to deal with the large size of this dataset, any challenges you faced and how you resolved them. [No more than 200 words]**

1) The strategy employed to handle the large dataset was chunked processing using pandas.read_csv with the chunksize parameter. This approach divided the dataset into manageable chunks of 10,000 rows, enabling processing without exceeding memory limits. Each chunk was cleaned, processed, and analyzed independently, allowing for iterative and memory-efficient operations.

2) Challenges and Solutions:

- *Data Cleaning:* The dataset contained null bytes (\0), which caused reading errors. These were resolved by preprocessing the file line by line to remove null bytes before loading the data.

- *Malformed Rows:* Certain rows did not conform to the expected structure, leading to parsing issues. This was addressed by using on_bad_lines='skip', ensuring faulty rows were ignored.

- *Date Parsing:* The ACTIVITY_TIME column had complex formats, making parsing challenging. Using pd.to_datetime with errors="coerce" ensured invalid entries were set to NaT, preventing processing failures.

- *Aggregation Across Chunks:* Combining results across chunks required consistent grouping logic. Grouping by year_month and leveraging pandas' aggregation functions provided a seamless way to find the period with the most events.

This strategy balances memory efficiency with the need to derive insights from large datasets.

# Question 2 - Standard Error of the Mean (SEM) with Bootstrapping


**Dataset:** hh_ml.zip

**Task 1: Calculate the SEM for age in the dataset. Please use no less than 100 bootstrap samples for your calculation. Write a function which takes innumber of  bootstrap samples and returns the SEM.**

**Useful librairies**

In [10]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
import joblib
from joblib import Memory
from datetime import datetime
from joblib import Parallel, delayed

**Extract the data and process**

In [11]:
!unzip '/content/drive/MyDrive/DATASETS/raw/hh_data_ml.zip' -d '/content'

Archive:  /content/drive/MyDrive/DATASETS/raw/hh_data_ml.zip
replace /content/hh_data_ml.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [12]:
file_path = ('/content/hh_data_ml.csv')
useful_col = ['P07M', 'P07A']

In [13]:
dataset = pd.read_csv(file_path, sep = '|', usecols = useful_col, chunksize = 1000)

**Compute the age of an individual based on their birth year and month.**

In [14]:
def age(birth_data):
    # Get the current date details
    today = datetime.today()
    birth_year = birth_data['P07A']
    birth_month = birth_data['P07M']

    # Start with a rough age calculation
    years_difference = today.year - birth_year

    # Check if the birth date is later in the year than today’s date
    if today.month < birth_month:
        years_difference -= 1

    return years_difference


**Compute an array of ages based on birth month and year columns in a DataFrame.**

In [15]:
def col_ages(df):
    # Get the current year and month
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month

    # Function to compute age for each row
    def com_age(row):
        year_diff = current_year - row['P07A']
        if current_month < row['P07M']:
            year_diff -= 1
        return year_diff

    # Calculate ages for all rows and convert to a NumPy array
    ages = np.array(df.apply(com_age, axis=1))

    return ages


In [16]:
dataset = pd.read_csv(file_path, sep = '|', usecols = useful_col, chunksize = 1000)
# Initialize an empty Python list to store all calculated ages.
ages_list = []

# Process the data chunk by chunk to efficiently handle large datasets.
for chunk in dataset:
    ages_list.extend(col_ages(chunk))

# Convert the list of ages to a NumPy array for further processing.
ages = np.array(ages_list)


**Perform bootstraping and compute mean**

In [17]:
# Configure caching for the function
memory = Memory(location='cache_directory', verbose=0)

# Define a bootstrapping function
@memory.cache
def compute_stat(data, n_samples):
    num_data_points = len(data)
    bootstrap_means = np.empty(n_samples, dtype=np.float32)

    # Perform bootstrapping and calculate means for each sample
    for sample_idx in range(n_samples):
        sampled_data = resample(data, n_samples=num_data_points, replace=True)
        bootstrap_means[sample_idx] = sampled_data.mean()

    return bootstrap_means


**Calculate the standard deviation of bootstrap sample means using parallel processing.**

In [28]:
from scipy.stats import sem

def bootstrap(data):
    # Number of bootstrap samples to draw in each job
    bootstrap_sizes = [10] * 10  # List with repeated sample size

    # Generate bootstrap sample means in parallel
    bootstrap_means = Parallel(n_jobs=-1)(
        delayed(compute_stat)(data, size) for size in bootstrap_sizes
    )

    # Return the standard deviation of the sample means
    #return np.std(bootstrap_means, dtype=np.float64, ddof = 1) / np.sqrt(len(bootstrap_means))
    return sem(np.append([],bootstrap_means))

def execute():
  bootstrap_std_dev = bootstrap(ages)

  # Display the calculated result
  return bootstrap_std_dev


**Use all of these functions**

In [29]:
execute()

0.03922975205306812

**task 2: Describe what strategy you used to deal with the fact that you have to run many computations for this task as well as the challenges you faced and how you solved those challenges. [No more than 200 words]**

*- Parallel Processing:*

  Used joblib.Parallel to distribute the computational workload across multiple CPU cores. This significantly reduced the execution time for generating bootstrap sample means by leveraging parallelism.

*- Caching:*

  Implemented caching with joblib.Memory to store results of repeated computations. This prevented recalculating identical operations and saved time by reusing previously computed results.

*- Chunk Processing:*

  Processed the data in smaller chunks to manage large datasets efficiently. This approach helped prevent memory overload and allowed for better management of system resources.

*- Memory Management:*

  Initially, I faced memory inefficiency using np.append to add results iteratively. I solved this by accumulating results in Python lists and only converting them into NumPy arrays after all computations were complete, improving memory handling.

*- Efficient Data Types:*

  To optimize memory usage, I used float32 for the bootstrap sample means instead of float64, which balanced memory consumption and computation accuracy.

# Question 3 - Weather Forecast for All Capital Cities in Africa


In [30]:
import requests
import json
import pandas as pd

**Import dataset**

In [31]:
file_path_africa_dataset = ('/content/drive/MyDrive/DATASETS/raw/Africa_Cities.csv')

In [32]:
# Load Africa Cities dataset

africa_dataset = pd.read_csv(file_path_africa_dataset)

**Filter the city column and the country column**

In [33]:
# Filter relevant columns

filtered_cities = africa_dataset[['CITY_NAME', 'CNTRY_NAME']].drop_duplicates()
list_city = filtered_cities['CITY_NAME'].tolist()
list_country = filtered_cities['CNTRY_NAME'].tolist()

**Define API key and the base url**

In [34]:
# My API KEY

API_KEY = 'f34edc75d372db75fb38555fd2a061d6'
base_url = 'http://api.openweathermap.org/data/2.5/'


**fetch_weather function is a function that take city and country as input, fetch the date concerning the weather in the website correspond to the city and country in africa cities.**

In [35]:
def fetch_weather(city, country):
  response = requests.get(f"{base_url}forecast?q={city},{country}&APPID={API_KEY}&units=metric") # Corrected to use the corrected base URL
  if response.status_code == 200:
        data = response.json()
        for forecast in data.get('list', []):
          forecast_date = datetime.utcfromtimestamp(forecast['dt']).strftime('%Y-%m-%d')
          if forecast_date == '2025-01-13':
            return {
                    'Country': country,
                    'City': city,
                    'Date': forecast_date,
                    'Time': datetime.utcfromtimestamp(forecast['dt']).strftime('%H:%M:%S'),
                    'Weather_main': forecast['weather'][0]['main'],
                    'Temp': forecast['main']['temp'],
                    'Temp_min': forecast['main']['temp_min'],
                    'Temp_max': forecast['main']['temp_max'],
                    'Humidity': forecast['main']['humidity'],
                    'Clouds': forecast['clouds']['all']
                }
  else:
    print(f"Error fetching weather data for {city}, {country}: {response.status_code}")
  return {}

In [36]:
fetch_weather("Accra", "Ghana")

{'Country': 'Ghana',
 'City': 'Accra',
 'Date': '2025-01-13',
 'Time': '00:00:00',
 'Weather_main': 'Clouds',
 'Temp': 27.97,
 'Temp_min': 27.44,
 'Temp_max': 27.97,
 'Humidity': 81,
 'Clouds': 21}

**Create a list that store the infos about the weather and make it as a dataframe**

In [37]:
df = []
for city, country in zip(list_city, list_country):
  dataframe = df.append(fetch_weather(city, country))

Error fetching weather data for Fderik, Mauritania: 404
Error fetching weather data for Sidi-Bel-Abbes, Algeria: 404
Error fetching weather data for Tidjikdja, Mauritania: 404
Error fetching weather data for Tingrela, Cote d'Ivoire: 404
Error fetching weather data for Mbahiakro, Cote d'Ivoire: 404
Error fetching weather data for Grand Lahou, Cote d'Ivoire: 404
Error fetching weather data for Ghardina, Algeria: 404
Error fetching weather data for Tbessa, Algeria: 404
Error fetching weather data for Medemine, Tunisia: 404
Error fetching weather data for L'Ariana, Tunisia: 404
Error fetching weather data for Sdid Bouzid, Tunisia: 404
Error fetching weather data for Monasir, Tunisia: 404
Error fetching weather data for El Minya, Egypt: 404
Error fetching weather data for El-Kharga, Egypt: 404
Error fetching weather data for Dumyat, Egypt: 404
Error fetching weather data for Kafr el Sheikh, Egypt: 404
Error fetching weather data for El Mansura, Egypt: 404
Error fetching weather data for Shi

In [38]:
filename = 'weather_data.csv'

col_df = ["Country", "City", "Date", "Time", "Weather_main", "Temp", "Temp_min", "Temp_max", "Humidity", "Clouds"]
df = pd.DataFrame(df, columns = col_df)
df.to_csv(filename, index=False)

In [39]:
weather_data = pd.read_csv(filename)
weather_data.head()

,Country,City,Date,Time,Weather_main,Temp,Temp_min,Temp_max,Humidity,Clouds
0,St. Helena,Jamestown,2025-01-13,00:00:00,Clouds,-1.86,-3.54,-1.86,81.0,83.0
1,Cape Verde,Praia,2025-01-13,00:00:00,Clear,23.07,22.61,23.07,67.0,0.0
2,Spain,Santa Cruz de Tenerife,2025-01-13,00:00:00,Rain,18.64,17.99,18.64,73.0,76.0
3,Spain,Las Palmas,2025-01-13,00:00:00,Clouds,17.15,16.56,17.15,75.0,61.0
4,Portugal,Funchal,2025-01-13,00:00:00,Clouds,18.76,17.32,18.76,88.0,61.0
